In [34]:
import selenium
from selenium import webdriver
import os, time
wd = webdriver.Chrome()

In [26]:
os.chdir('/Users/jiafengchen/Downloads/')

In [29]:
years = ["2012-2020",
"2002-2010",
"1992-2000",
"1982-1990",
"1972-1980",
"1968-1970"]
states = ["1","10","17","12","24","81","88","95","97",
          "121","133","143","146","167","136","179","184","191","199",
          "221","210","230","247","266","256","272","289","314","293","296","310","317",
          "274","287","349","369","376","382","403","406","498","413","423","454","470","458",
          "472","492","482","496",]

In [35]:
for y in tqdm(years):
    for s in states:
        url = "http://library.cqpress.com.ezp-prod1.hul.harvard.edu/elections/download-data-results.php?filetype=1&office=3&areatype=3&year={}&states%5B%5D={}&format=3&license=on&emailto=&emailfrom=&filename=\"file.csv\"".format(y,s)
        wd.get(url)
        while 'export.csv' not in os.listdir():
            time.sleep(1)
        os.rename('export.csv', 'export_{}_{}.csv'.format(y,s))

In [69]:
df = []
for file in os.listdir():
    if 'csv' in file:
        df.append(pd.read_csv(file, header=2, encoding='latin-1', 
                              thousands=',').dropna(how='all'))


In [80]:
df = pd.concat(df)
df['RedistrictedDate'] = pd.to_datetime(df['RedistrictedDate'].apply(lambda x: str(int(x)) if not np.isnan(x) else x))
df = df.dropna(axis=1, how='all')
df = df.reset_index(drop=True).drop('Office', axis=1)
df.columns = [s.lower() for s in df.columns]

In [101]:
up = df['repvotes'] == 'Unopposed'
df.loc[up, 'rep_unopposed'] = True
df.loc[up, 'repvotes'] = np.nan

up = df['demvotes'] == 'Unopposed'
df.loc[up, 'dem_unopposed'] = True
df.loc[up, 'demvotes'] = np.nan


In [106]:
def clean_number(x):
    if type(x) is str:
        return float(x.replace(',', ''))
    return x

In [108]:
df['repvotes'] = df['repvotes'].apply(clean_number)
df['demvotes'] = df['demvotes'].apply(clean_number)

In [110]:
df.sort_values(['state','raceyear'], ascending=[True,False]).reset_index(drop=True).to_feather('/Users/jiafengchen/Documents/Senior/Stat 195/project/past_house.feather')